# The NMT model based on PyTorch Seq2Seq Tutorial


## Preparing Data

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator, TabularDataset 

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import numpy as np

import random
import math
import time
import pandas as pd

Next, we'll set the random seed for reproducability.

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

When using packed padded sequences, we need to tell PyTorch how long the actual (non-padded) sequences are. Luckily for us, TorchText's `Field` objects allow us to use the `include_lengths` argument, this will cause our `batch.src` to be a tuple. The first element of the tuple is the same as before, a batch of numericalized source sentence as a tensor, and the second element is the non-padded lengths of each source sentence within the batch.

#### Load the data

In [3]:
data_path = 'dict_data'  # news_data or dict_data

def tokenize(word): # create a tokenizer function
    word = word.replace('\n', '')
    return word.split(' ')

# <sos>: start of a sequence; <eos>: end of a sequence.
SRC = Field(tokenize=tokenize, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True,
            include_lengths = True)

TRG = Field(tokenize=tokenize, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=False,
            include_lengths = True)

PINYIN_STR = Field(tokenize=tokenize, 
                   init_token='<sos>', 
                   eos_token='<eos>', 
                   lower=True,
                   include_lengths = True)

PINYIN_CHAR = Field(tokenize=tokenize, 
                    init_token='<sos>', 
                    eos_token='<eos>', 
                    lower=True,
                    include_lengths = True)

train_data = TabularDataset(
           path = data_path + "/train.tsv", 
           format='tsv',
           skip_header=True, 
           fields=([("src", SRC), ("trg", TRG), ("pinyin_str", PINYIN_STR), ("pinyin_char", PINYIN_CHAR)]))

valid_data = TabularDataset(
           path = data_path + "/valid.tsv", 
           format='tsv',
           skip_header=True, 
           fields=([("src", SRC), ("trg", TRG), ("pinyin_str", PINYIN_STR), ("pinyin_char", PINYIN_CHAR)]))

test_data = TabularDataset(
           path = data_path + "/test.tsv",   #/dev.tsv for news_data
           format='tsv',
           skip_header=True, 
           fields=([("src", SRC), ("trg", TRG), ("pinyin_str", PINYIN_STR), ("pinyin_char", PINYIN_CHAR)]))

In [4]:
SRC.build_vocab(train_data, min_freq=1)
TRG.build_vocab(train_data, min_freq=1)  # first hyperparameter, the min_freq for vocab
PINYIN_STR.build_vocab(train_data, min_freq=1)
PINYIN_CHAR.build_vocab(train_data, min_freq=1)

Next, we handle the iterators.

One quirk about packed padded sequences is that all elements in the batch need to be sorted by their non-padded lengths in descending order, i.e. the first sentence in the batch needs to be the longest. We use two arguments of the iterator to handle this, `sort_within_batch` which tells the iterator that the contents of the batch need to be sorted, and `sort_key` a function which tells the iterator how to sort the elements in the batch. Here, we sort by the length of the `src` sentence.

In [5]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x : len(x.src),
     device = device)

cuda


## Building the Model

### Encoder

Next up, we define the encoder.

The changes here all within the `forward` method. It now accepts the lengths of the source sentences as well as the sentences themselves. 

After the source sentence (padded automatically within the iterator) has been embedded, we can then use `pack_padded_sequence` on it with the lengths of the sentences. `packed_embedded` will then be our packed padded sequence. This can be then fed to our RNN as normal which will return `packed_outputs`, a packed tensor containing all of the hidden states from the sequence, and `hidden` which is simply the final hidden state from our sequence. `hidden` is a standard tensor and not packed in any way, the only difference is that as the input was a packed sequence, this tensor is from the final **non-padded element** in the sequence.

We then unpack our `packed_outputs` using `pad_packed_sequence` which returns the `outputs` and the lengths of each, which we don't need. 

The first dimension of `outputs` is the padded sequence lengths however due to using a packed padded sequence the values of tensors when a padding token was the input will be all zeros.

In [6]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, emb_dropout, num_layers, rnn_type='GRU', rnn_dropout=0.2):
        super().__init__()
        
        self.enc_hid_dim = enc_hid_dim
        self.rnn_type = rnn_type
        self.cell = None  # For decoder LSTM init cell state
        
        # input_dim is the vocab_size, i.e. the emb_dim of one-hot encoding
        self.embedding = nn.Sequential(
            nn.Embedding(input_dim, emb_dim),
            nn.Dropout(emb_dropout)
        )
        
        if rnn_type == 'GRU':
            self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True, num_layers=num_layers, dropout=rnn_dropout)
        elif rnn_type == 'LSTM':
            self.rnn = nn.LSTM(emb_dim, enc_hid_dim, bidirectional = True, num_layers=num_layers, dropout=rnn_dropout)
        
        
    def forward(self, src, src_len):
        
        # src = [src len, batch size]
        # src_len = [src len]
        
        embedded = self.embedding(src)
        
        # embedded = [src len, batch size, emb dim]
                
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len)
        
        packed_outputs, hidden = self.rnn(packed_embedded)
        if self.rnn_type == 'LSTM':
            hidden, cell = hidden[0], hidden[1]  # (h_n, c_n)
            self.cell = torch.cat((cell[-2,:,:], cell[-1,:,:]), dim = 1)
                                 
        # packed_outputs is a packed sequence containing all hidden states
        # hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
            
        # outputs is now a non-packed sequence, all hidden states obtained
        #  when the input is a pad token are all zeros
            
        # outputs = [src len, batch size, hid dim * num directions]
        # hidden = [n layers * num directions, batch size, hid dim]
        
        # hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        # outputs are always from the last layer
        
        # hidden [-2, :, : ] is the last of the forwards RNN 
        # hidden [-1, :, : ] is the last of the backwards RNN
        
        # initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        # hidden = self.layerNorm(hidden)
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

### Attention

The attention module is where we calculate the attention values over the source sentence. 

Previously, we allowed this module to "pay attention" to padding tokens within the source sentence. However, using *masking*, we can force the attention to only be over non-padding elements.

The `forward` method now takes a `mask` input. This is a **[batch size, source sentence length]** tensor that is 1 when the source sentence token is not a padding token, and 0 when it is a padding token. For example, if the source sentence is: ["hello", "how", "are", "you", "?", `<pad>`, `<pad>`], then the mask would be [1, 1, 1, 1, 1, 0, 0].

We apply the mask after the attention has been calculated, but before it has been normalized by the `softmax` function. It is applied using `masked_fill`. This fills the tensor at each element where the first argument (`mask == 0`) is true, with the value given by the second argument (`-1e10`). In other words, it will take the un-normalized attention values, and change the attention values over padded elements to be `-1e10`. As these numbers will be miniscule compared to the other values they will become zero when passed through the `softmax` layer, ensuring no attention is payed to padding tokens in the source sentence.

In [7]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        self.linear_out = nn.Linear(enc_hid_dim * 2, dec_hid_dim, bias=True)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
  
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention = [batch size, src len]
        
        attention = attention.masked_fill(mask == 0, -1e10)
        
        a = F.softmax(attention, dim = 1).unsqueeze(1)  # a = [batch size, 1, src len]     
        weighted = torch.bmm(a, encoder_outputs) # weighted = [batch size, 1, enc hid dim * 2]  
        weighted = weighted.permute(1, 0, 2) # weighted = [1, batch size, enc hid dim * 2]
        
        return a, self.linear_out(weighted)

### Decoder

The decoder only needs a few small changes. It needs to accept a mask over the source sentence and pass this to the attention module. As we want to view the values of attention during inference, we also return the attention tensor.

In [8]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, emb_dropout, 
                 attention, num_layers, rnn_type='GRU', rnn_dropout=0.2):
        super().__init__()

        self.output_dim = output_dim
        self.dec_hid_dim = dec_hid_dim
        self.num_layers = num_layers
        self.attention = attention
        self.rnn_type = rnn_type
        
        # self.layerNorm = nn.LayerNorm(dec_hid_dim)
        
        self.embedding = nn.Sequential(
            nn.Embedding(output_dim, emb_dim),
            nn.Dropout(emb_dropout)
        )
        
        if rnn_type == 'GRU':
            self.rnn = nn.GRU(emb_dim + dec_hid_dim, dec_hid_dim, num_layers=num_layers, dropout=rnn_dropout)
        elif rnn_type == 'LSTM':
            self.rnn = nn.LSTM(emb_dim + dec_hid_dim, dec_hid_dim, num_layers=num_layers, dropout=rnn_dropout)
            self.cell = None
            
        self.pred = nn.Sequential(
            nn.Linear(2 * dec_hid_dim + emb_dim, output_dim),
            nn.LogSoftmax(dim = 1)
        )
        
        
    def forward(self, input, hidden, encoder_outputs, mask, cell=None):
             
        # input = [batch size]
        # hidden = [batch size, dec hid dim], cell for LSTM only
        # encoder_outputs = [src len, batch size, enc hid dim * 2]
        # mask = [batch size, src len]
        
        input = input.unsqueeze(0)  # input = [1, batch size]
        embedded = self.embedding(input)  #  embedded = [1, batch size, emb dim]
        a, attn_output = self.attention(hidden, encoder_outputs, mask)  # attn computed with last hidden and encoder_outputs

        #print(hidden.shape)
        if self.rnn_type =='GRU':
            output, hidden = self.rnn(torch.cat((embedded, attn_output), dim=2), hidden.unsqueeze(0).repeat(2, 1, 1))
        elif self.rnn_type == 'LSTM':
            output, hidden_cell = self.rnn(torch.cat((embedded, attn_output), dim=2), (hidden.unsqueeze(0).repeat(2, 1, 1), 
                                                                                       cell.unsqueeze(0).repeat(2, 1, 1)))
            hidden, cell = hidden_cell[0], hidden_cell[1]
            self.cell = cell[-1]  # last layer cell
        hidden = hidden.view(self.num_layers, 1, embedded.size(1), self.dec_hid_dim)[-1]  # last layer hidden extracted
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        attn_output = attn_output.squeeze(0)
        
        prediction = self.pred(torch.cat((output, attn_output, embedded), dim = 1))  #  prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

In [9]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder_list, src_pad_idx, device, teacher_forcing_ratio = 0.5):
        super().__init__()
        
        self.encoder = encoder
            
        # decoder_list has one decoder or two depending on the model used
        self.decoder = decoder_list[0]
        if len(decoder_list) > 1:
            self.multi_task = True
            self.decoder_pinyin = decoder_list[1]
        else:
            self.multi_task=False
        
        # bridge layer from enc to dec
        self.bridge = nn.Linear(self.encoder.enc_hid_dim * 2, self.decoder.dec_hid_dim)
        if self.decoder.rnn_type == 'LSTM':
            self.bridge_cell = nn.Linear(self.encoder.enc_hid_dim * 2, self.decoder.dec_hid_dim)
        
        if self.multi_task:
            self.bridge_pinyin = nn.Linear(self.encoder.enc_hid_dim * 2, self.decoder_pinyin.dec_hid_dim)
            if self.decoder.rnn_type == 'LSTM':
                self.bridge_cell_pinyin = nn.Linear(self.encoder.enc_hid_dim * 2, self.decoder_pinyin.dec_hid_dim)
                
        self.src_pad_idx = src_pad_idx
        self.device = device
        self.teacher_forcing_ratio = teacher_forcing_ratio 
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
    
    def forward_pinyin(self, src, src_len, hidden_pinyin, encoder_outputs, pinyin):
        assert self.multi_task == True
        # hidden_pinin is pre-calculated in the main decoder
        
        batch_size = src.shape[1]
        pinyin_len = pinyin.shape[0]
        pinyin_vocab_size = self.decoder_pinyin.output_dim
        outputs_pinyin = torch.zeros(pinyin_len, batch_size, pinyin_vocab_size).to(self.device)
        
        cell_pinyin = None
        if self.decoder.rnn_type == 'LSTM':
            cell_pinyin = torch.tanh(self.bridge_cell_pinyin(self.encoder.cell))
            
        input_pinyin = pinyin[0, :]
        mask = self.create_mask(src)
        
        for t in range(1, pinyin_len): 

            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden_pinyin, _ = self.decoder_pinyin(input_pinyin, hidden_pinyin, encoder_outputs, mask, cell=cell_pinyin)
            cell_pinyin = self.decoder_pinyin.cell

            #place predictions in a tensor holding predictions for each token
            outputs_pinyin[t] = output

            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < self.teacher_forcing_ratio

            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input_pinyin = pinyin[t] if teacher_force else top1
            
        return outputs_pinyin
        
        
    def forward(self, src, src_len, trg, pinyin = None):
        
        # src = [src len, batch size]
        # src_len = [batch size]
        # trg = [trg len, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
                    
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # encoder_outputs is all hidden states of the input sequence, back and forwards
        # hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_len)
        hidden_trg = torch.tanh(self.bridge(hidden))
        if self.multi_task:
            hidden_pinyin = torch.tanh(self.bridge_pinyin(hidden))
            
        # cell states for LSTM
        cell_trg = None
        if self.decoder.rnn_type == 'LSTM':
            cell_trg = torch.tanh(self.bridge_cell(self.encoder.cell))
            
                
        # first input to the decoder is the <sos> tokens
        input_trg = trg[0, :]
        
        mask = self.create_mask(src)

        #mask = [batch size, src len]
                
        for t in range(1, trg_len): 
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden_trg, _ = self.decoder(input_trg, hidden_trg, encoder_outputs, mask, cell=cell_trg)
            cell_trg = self.decoder.cell
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < self.teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input_trg = trg[t] if teacher_force else top1
            
        if self.multi_task:
            outputs_pinyin = self.forward_pinyin(src, src_len, hidden_pinyin, encoder_outputs, pinyin=pinyin)
            return outputs, outputs_pinyin
        
        return outputs

## Training the Seq2Seq Model

Next up, initializing the model and placing it on the GPU.

In [10]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
OUTPUT_DIM_str = len(PINYIN_STR.vocab)
OUTPUT_DIM_char = len(PINYIN_CHAR.vocab)
ENC_EMB_DIM = 256  # 512 in paper, 256 best
DEC_EMB_DIM = 256
DEC_EMB_DIM2 = 128
ENC_HID_DIM = 512  # 1024 in paper, 512 best
DEC_HID_DIM = 512
DEC_HID_DIM2 = 256
ENC_DROPOUT = 0.1  # 0.1 in paper, for embedding
DEC_DROPOUT = 0.1  # 0.1 in paper, for embedding
RNN_DROPOUT = 0.2
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
RNN_TYPE = 'LSTM'

encoder_num_layer = 2
decoder_num_layer = 2

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
attn2 = Attention(ENC_HID_DIM, DEC_HID_DIM2)

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, ENC_DROPOUT, encoder_num_layer, 
              rnn_type=RNN_TYPE, rnn_dropout=RNN_DROPOUT)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, 
              attn, decoder_num_layer, rnn_type='LSTM', rnn_dropout=RNN_DROPOUT)
dec2 = Decoder(OUTPUT_DIM_str, DEC_EMB_DIM2, ENC_HID_DIM, DEC_HID_DIM2, DEC_DROPOUT, 
               attn2, decoder_num_layer, rnn_type=RNN_TYPE, rnn_dropout=RNN_DROPOUT)

dec_list = [dec, dec2]
# dec_list = [dec]

model = Seq2Seq(enc, dec_list, SRC_PAD_IDX, device).to(device)

Then, we initialize the model parameters.

In [11]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
show_params = model.apply(init_weights)
show_params

Seq2Seq(
  (encoder): Encoder(
    (embedding): Sequential(
      (0): Embedding(30, 256)
      (1): Dropout(p=0.1, inplace=False)
    )
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.2, bidirectional=True)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
      (linear_out): Linear(in_features=1024, out_features=512, bias=True)
    )
    (embedding): Sequential(
      (0): Embedding(437, 256)
      (1): Dropout(p=0.1, inplace=False)
    )
    (rnn): LSTM(768, 512, num_layers=2, dropout=0.2)
    (pred): Sequential(
      (0): Linear(in_features=1280, out_features=437, bias=True)
      (1): LogSoftmax()
    )
  )
  (decoder_pinyin): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1280, out_features=256, bias=True)
      (v): Linear(in_features=256, out_features=1, bias=False)
      (linear_out): Linear(in_features=1024, out_featu

We'll print out the number of trainable parameters in the model, noticing that it has the exact same amount of parameters as the model without these improvements.

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 19,728,323 trainable parameters


Then we define our optimizer and criterion. 

The `ignore_index` for the criterion needs to be the index of the pad token for the target language, not the source language.

In [13]:
learning_rate = 0.003  # 0.003 in paper
patience = 0
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer=optimizer,
                mode='max', factor=0.9, # 0.9 in paper
                patience=patience)

In [14]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

Next, we'll define our training and evaluation loops.

As we are using `include_lengths = True` for our source field, `batch.src` is now a tuple with the first element being the numericalized tensor representing the sentence and the second element being the lengths of each sentence within the batch.

Our model also returns the attention vectors over the batch of source source sentences for each decoding time-step. We won't use these during the training/evaluation, but we will later for inference.

In [15]:
def update(epoch, valid_loss, valid_acc, 
           best_valid_loss, best_valid_acc, acc_valid_loss,
           update_type='acc'):
    global best_valid_epoch, early_stop_patience, full_patience, best_train_step, train_steps, exp_num
    print("\n---------------------------------------")
    print("[Epoch: {}][Validatiing...]".format(epoch))
    if valid_loss < best_valid_loss:
        print('\t\t Better Valid Loss!')
        best_valid_loss = valid_loss
        if update_type == 'loss':
            torch.save(model.state_dict(), 'loss-model.pt')
        early_stop_patience = full_patience  # restore full patience if obtain new minimum of the loss
    else:
        if early_stop_patience > 0:
            early_stop_patience += -1
    
    if valid_acc > best_valid_acc or (valid_acc == best_valid_acc and valid_loss < acc_valid_loss):
        print('\t\t Better Valid Acc!')
        best_valid_acc = valid_acc
        acc_valid_loss = valid_loss
        best_valid_epoch = epoch
        best_train_step = train_steps
        if update_type == 'acc':
            torch.save(model.state_dict(), 'experiments/exp' + str(exp_num) + '/acc-model-seq2seq.pt')
    print(f'\t patience: {early_stop_patience}/{full_patience}')
    print(f'\t Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc:.3f} | Val. PPL: {math.exp(valid_loss):7.3f}')
    print(f'\t BEST. Val. Loss: {best_valid_loss:.3f} | BEST. Val. Acc: {best_valid_acc:.3f} | Val. Loss: {acc_valid_loss:.3f} | BEST. Val. Epoch: {best_valid_epoch} | BEST. Val. Step: {best_train_step}')
    print("---------------------------------------\n")
    return best_valid_loss, best_valid_acc, acc_valid_loss

In [16]:
n_examples = len(train_data.examples)

def train(model, iterator, 
          optimizer, criterion, 
          clip, epoch,
          scheduler, valid_iterator,
          multi_task = False, multi_task_ratio=0.5):
    
    global best_valid_loss, acc_valid_loss, best_valid_acc, best_valid_epoch, train_steps, report_steps, tfr
    model.train()
    model.teacher_forcing_ratio = tfr
    print("[Train]: Current Teacher Forcing Ratio: {:.3f}".format(model.teacher_forcing_ratio))
    
    epoch_loss = 0
    running_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src, src_len = batch.src
        trg, trg_len = batch.trg
        pinyin, pinyin_len = batch.pinyin_str
        
        optimizer.zero_grad()
        
        if multi_task:
            output, output_pinyin = model(src, src_len, trg, pinyin=pinyin)
        else:
            output = model(src, src_len, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        if multi_task:
            output_pinyin_dim = output_pinyin.shape[-1]
            output_pinyin = output_pinyin[1:].view(-1, output_pinyin_dim)
            pinyin = pinyin[1:].view(-1)
            loss = (multi_task_ratio * criterion(output, trg)) + ((1 - multi_task_ratio) * criterion(output_pinyin, pinyin))
        else:
            loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        running_loss = epoch_loss / (i + 1)
        
        # print every 50 batches (50 steps)
        if i % report_steps == report_steps - 1:
            train_steps += report_steps  # by doing so, the last batch is neglected
            for param_group in optimizer.param_groups:
                lr = param_group['lr']
            print('[Epoch: {}][#examples: {}/{}][#steps: {}]'.format(epoch, (i+1) * BATCH_SIZE, n_examples, train_steps))
            print(f'\tTrain Loss: {running_loss:.3f} | Train PPL: {math.exp(running_loss):7.3f} | lr: {lr:.3e}')
            
            # eval the validation set for every * steps
            if (train_steps % (10 * report_steps)) == 0:
                print('-----val------')
                valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, scheduler, multi_task = multi_task)
                print('-----tst------')
                test_loss, test_acc = evaluate(model, test_iterator, criterion, scheduler, is_test=True, multi_task=multi_task)
                best_valid_loss, best_valid_acc, acc_valid_loss = update(epoch, valid_loss, valid_acc, 
                                                         best_valid_loss, best_valid_acc, acc_valid_loss,
                                                         update_type='acc')
                scheduler.step(valid_acc)  # must be placed here otherwise the test acc messes up
                model.train()
                
            
    return epoch_loss / len(iterator)

In [17]:
def evaluate(model, iterator, criterion, scheduler, is_test=False, multi_task=False, multi_task_ratio=0.5):
    
    model.eval()
    model.teacher_forcing_ratio = 0 #  turn off teacher forcing
    print("[Eval Start]: Current Teacher Forcing Ratio: {:.3f}".format(model.teacher_forcing_ratio))
    
    epoch_loss = 0
    correct = 0
    correct_pinyin = 0
    
    global valid_data, test_data, tfr
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, src_len = batch.src
            trg, trg_len = batch.trg
            pinyin, pinyin_len = batch.pinyin_str

            if multi_task:
                output, output_pinyin = model(src, src_len, trg, pinyin=pinyin)
            else:
                output = model(src, src_len, trg)
            
            # ---------compute acc START----------
            pred = output[1:].argmax(2).permute(1, 0) # [batch_size, trg_len]
            ref = trg[1:].permute(1, 0)
            # consider the last batch as well
            size = pred.shape[0]
            for j in range(size):
                
                pred_j = pred[j, :]
                pred_j_toks = []
                for t in pred_j:
                    tok = TRG.vocab.itos[t]
                    if tok == '<eos>':
                        break
                    else:
                        pred_j_toks.append(tok)
                pred_j = ''.join(pred_j_toks)
                
                ref_j = ref[j, :]
                ref_j_toks = []
                for t in ref_j:
                    tok = TRG.vocab.itos[t]
                    if tok == '<eos>':
                        break
                    else:
                        ref_j_toks.append(tok)
                ref_j = ''.join(ref_j_toks)
                
                if pred_j == ref_j:
                    correct += 1
            # ---------compute acc END----------
            
            # ---------compute acc START----------
            if multi_task:
                pred_pinyin = output_pinyin[1:].argmax(2).permute(1, 0) # [batch_size, pinyin_len]
                ref_pinyin = pinyin[1:].permute(1, 0)
                # consider the last batch as well
                size = pred_pinyin.shape[0]
                for j in range(size):

                    pred_j = pred_pinyin[j, :]
                    pred_j_toks = []
                    for t in pred_j:
                        tok = PINYIN_STR.vocab.itos[t]
                        if tok == '<eos>':
                            break
                        else:
                            pred_j_toks.append(tok)
                    pred_j = ''.join(pred_j_toks)

                    ref_j = ref_pinyin[j, :]
                    ref_j_toks = []
                    for t in ref_j:
                        tok = PINYIN_STR.vocab.itos[t]
                        if tok == '<eos>':
                            break
                        else:
                            ref_j_toks.append(tok)
                    ref_j = ''.join(ref_j_toks)
                    
                    if pred_j == ref_j:
                        correct_pinyin += 1
                # ---------compute acc END----------
            
            
            # trg = [trg len, batch size]
            # output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            if multi_task:
                output_pinyin_dim = output_pinyin.shape[-1]
                output_pinyin = output_pinyin[1:].view(-1, output_pinyin_dim)
                pinyin = pinyin[1:].view(-1)
                loss = (multi_task_ratio * criterion(output, trg)) + ((1 - multi_task_ratio) * criterion(output_pinyin, pinyin))
            else:
                loss = criterion(output, trg)

            epoch_loss += loss.item()
        
        # compute loss and acc
        epoch_loss = epoch_loss / len(iterator)
        # sheduler applies on acc
        if not is_test:
            acc = correct / len(valid_data.examples)
            if multi_task:
                acc_pinyin = correct_pinyin / len(valid_data.examples)
                print('The number of correct pinyin predictions: {}'.format(correct_pinyin))
                print('Val Accuracy on Pinyin: {:.3f}'.format(acc_pinyin))
        else:
            acc = correct / len(test_data.examples)
            if multi_task:
                acc_pinyin = correct_pinyin / len(test_data.examples)
                print('The number of correct pinyin predictions: {}'.format(correct_pinyin))
                print('Test Accuracy on Pinyin: {:.3f}'.format(acc_pinyin))
        
        print('The number of correct predictions: {}'.format(correct))
        
        model.teacher_forcing_ratio = tfr  # restore teacher-forcing ratio
        print("[Eval End]: Current Teacher Forcing Ratio: {:.3f}".format(model.teacher_forcing_ratio))
    
    return epoch_loss, acc

In [18]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [19]:
N_EPOCHS = 100
CLIP = 1

best_valid_loss = float('inf')
acc_valid_loss = float('inf')
best_valid_acc = float(-1)
best_valid_epoch = -1
best_train_step = -1
multi_task = True
full_patience = 20
early_stop_patience = full_patience
train_steps = 0
report_steps = 50
multi_task_ratio = 0.75
exp_num = 1

try:
    for epoch in range(N_EPOCHS):
        
        if epoch <= 15:
            early_stop_patience = full_patience
        
        if early_stop_patience == 0:
            print("Early Stopping!")
            # break
            # abandon early stopping because we found best epoch in a long run
            
        start_time = time.time()
        
        tfr = max(1 - (float(10 + epoch * 1.5) / 50), 0.2) 
            
        train_loss = train(model, train_iterator, optimizer, criterion, CLIP, epoch, scheduler, valid_iterator,
                           multi_task=multi_task, multi_task_ratio=multi_task_ratio)
        
        valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, scheduler, is_test=False, 
                                         multi_task=multi_task, multi_task_ratio=multi_task_ratio)
        #test_loss, test_acc = evaluate(model, test_iterator, criterion, scheduler, is_test=True, multi_task=multi_task)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        best_valid_loss, best_valid_acc, acc_valid_loss = update(epoch, valid_loss, valid_acc, 
                                                 best_valid_loss, best_valid_acc, acc_valid_loss, update_type='acc')

        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc:.3f} | Val. PPL: {math.exp(valid_loss):7.3f}')
        # print(f'\t Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} | Test ACC: {test_acc:.3f}')
except KeyboardInterrupt:
        print("Exiting loop")

[Train]: Current Teacher Forcing Ratio: 0.800
[Epoch: 0][#examples: 3200/46620][#steps: 50]
	Train Loss: 4.022 | Train PPL:  55.829 | lr: 3.000e-03
[Epoch: 0][#examples: 6400/46620][#steps: 100]
	Train Loss: 3.773 | Train PPL:  43.513 | lr: 3.000e-03
[Epoch: 0][#examples: 9600/46620][#steps: 150]
	Train Loss: 3.520 | Train PPL:  33.798 | lr: 3.000e-03
[Epoch: 0][#examples: 12800/46620][#steps: 200]
	Train Loss: 3.226 | Train PPL:  25.189 | lr: 3.000e-03
[Epoch: 0][#examples: 16000/46620][#steps: 250]
	Train Loss: 2.915 | Train PPL:  18.444 | lr: 3.000e-03
[Epoch: 0][#examples: 19200/46620][#steps: 300]
	Train Loss: 2.638 | Train PPL:  13.987 | lr: 3.000e-03
[Epoch: 0][#examples: 22400/46620][#steps: 350]
	Train Loss: 2.403 | Train PPL:  11.057 | lr: 3.000e-03
[Epoch: 0][#examples: 25600/46620][#steps: 400]
	Train Loss: 2.213 | Train PPL:   9.143 | lr: 3.000e-03
[Epoch: 0][#examples: 28800/46620][#steps: 450]
	Train Loss: 2.050 | Train PPL:   7.769 | lr: 3.000e-03
[Epoch: 0][#examples: 

The number of correct pinyin predictions: 3962
Val Acc on Pinyin: 0.680
The number of correct predictions: 3777
[Eval End]: Current Teacher Forcing Ratio: 0.740

---------------------------------------
[Epoch: 2][Validatiing...]
		 Better Valid Loss!
		 Better Valid Acc!
	 patience: 20/20
	 Val. Loss: 0.490 | Val. Acc: 0.656 | Val. PPL:   1.633
	 BEST. Val. Loss: 0.490 | BEST. Val. Acc: 0.656 | Val. Loss: 0.490 | BEST. Val. Epoch: 2 | BEST. Val. Step: 2000
---------------------------------------

[Epoch: 2][#examples: 41600/46620][#steps: 2050]
	Train Loss: 0.358 | Train PPL:   1.431 | lr: 3.000e-03
[Epoch: 2][#examples: 44800/46620][#steps: 2100]
	Train Loss: 0.360 | Train PPL:   1.433 | lr: 3.000e-03
[Eval Start]: Current Teacher Forcing Ratio: 0.000
The number of correct pinyin predictions: 3955
Val Acc on Pinyin: 0.679
The number of correct predictions: 3752
[Eval End]: Current Teacher Forcing Ratio: 0.740

---------------------------------------
[Epoch: 2][Validatiing...]
	 patien

[Epoch: 5][#examples: 9600/46620][#steps: 3650]
	Train Loss: 0.210 | Train PPL:   1.234 | lr: 3.000e-03
[Epoch: 5][#examples: 12800/46620][#steps: 3700]
	Train Loss: 0.214 | Train PPL:   1.238 | lr: 3.000e-03
[Epoch: 5][#examples: 16000/46620][#steps: 3750]
	Train Loss: 0.217 | Train PPL:   1.242 | lr: 3.000e-03
[Epoch: 5][#examples: 19200/46620][#steps: 3800]
	Train Loss: 0.221 | Train PPL:   1.247 | lr: 3.000e-03
[Epoch: 5][#examples: 22400/46620][#steps: 3850]
	Train Loss: 0.224 | Train PPL:   1.251 | lr: 3.000e-03
[Epoch: 5][#examples: 25600/46620][#steps: 3900]
	Train Loss: 0.226 | Train PPL:   1.253 | lr: 3.000e-03
[Epoch: 5][#examples: 28800/46620][#steps: 3950]
	Train Loss: 0.228 | Train PPL:   1.257 | lr: 3.000e-03
[Epoch: 5][#examples: 32000/46620][#steps: 4000]
	Train Loss: 0.229 | Train PPL:   1.258 | lr: 3.000e-03
-----val------
[Eval Start]: Current Teacher Forcing Ratio: 0.000
The number of correct pinyin predictions: 4066
Val Acc on Pinyin: 0.698
The number of correct p

[Epoch: 7][#examples: 41600/46620][#steps: 5550]
	Train Loss: 0.159 | Train PPL:   1.172 | lr: 2.430e-03
[Epoch: 7][#examples: 44800/46620][#steps: 5600]
	Train Loss: 0.160 | Train PPL:   1.173 | lr: 2.430e-03
[Eval Start]: Current Teacher Forcing Ratio: 0.000
The number of correct pinyin predictions: 4106
Val Acc on Pinyin: 0.705
The number of correct predictions: 3934
[Eval End]: Current Teacher Forcing Ratio: 0.590

---------------------------------------
[Epoch: 7][Validatiing...]
	 patience: 17/20
	 Val. Loss: 0.520 | Val. Acc: 0.675 | Val. PPL:   1.682
	 BEST. Val. Loss: 0.488 | BEST. Val. Acc: 0.683 | Val. Loss: 0.520 | BEST. Val. Epoch: 7 | BEST. Val. Step: 5000
---------------------------------------

Epoch: 08 | Time: 1m 24s
	Train Loss: 0.161 | Train PPL:   1.174
	 Val. Loss: 0.520 | Val. Acc: 0.675 | Val. PPL:   1.682
[Train]: Current Teacher Forcing Ratio: 0.560
[Epoch: 8][#examples: 3200/46620][#steps: 5650]
	Train Loss: 0.105 | Train PPL:   1.110 | lr: 2.430e-03
[Epoch: 

[Epoch: 10][#examples: 25600/46620][#steps: 7400]
	Train Loss: 0.078 | Train PPL:   1.081 | lr: 2.187e-03
[Epoch: 10][#examples: 28800/46620][#steps: 7450]
	Train Loss: 0.079 | Train PPL:   1.082 | lr: 2.187e-03
[Epoch: 10][#examples: 32000/46620][#steps: 7500]
	Train Loss: 0.080 | Train PPL:   1.084 | lr: 2.187e-03
-----val------
[Eval Start]: Current Teacher Forcing Ratio: 0.000
The number of correct pinyin predictions: 4148
Val Acc on Pinyin: 0.712
The number of correct predictions: 3974
[Eval End]: Current Teacher Forcing Ratio: 0.500
-----tst------
[Eval Start]: Current Teacher Forcing Ratio: 0.000
The number of correct pinyin predictions: 4139
Val Acc on Pinyin: 0.710
The number of correct predictions: 3953
[Eval End]: Current Teacher Forcing Ratio: 0.500

---------------------------------------
[Epoch: 10][Validatiing...]
	 patience: 19/20
	 Val. Loss: 0.593 | Val. Acc: 0.682 | Val. PPL:   1.809
	 BEST. Val. Loss: 0.488 | BEST. Val. Acc: 0.687 | Val. Loss: 0.583 | BEST. Val. Epo

...

[Epoch: 89][#examples: 38400/46620][#steps: 62900]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 89][#examples: 41600/46620][#steps: 62950]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 89][#examples: 44800/46620][#steps: 63000]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
-----val------
[Eval Start]: Current Teacher Forcing Ratio: 0.000
The number of correct pinyin predictions: 4315
Val Acc on Pinyin: 0.740
The number of correct predictions: 4189
[Eval End]: Current Teacher Forcing Ratio: 0.200
-----tst------
[Eval Start]: Current Teacher Forcing Ratio: 0.000
The number of correct pinyin predictions: 4292
Val Acc on Pinyin: 0.736
The number of correct predictions: 4222
[Eval End]: Current Teacher Forcing Ratio: 0.200

---------------------------------------
[Epoch: 89][Validatiing...]
	 patience: 0/20
	 Val. Loss: 1.101 | Val. Acc: 0.719 | Val. PPL:   3.007
	 BEST. Val. Loss: 0.488 | BEST. Val. Acc: 0.720 | Val. Loss: 1.090 | BEST. Val. E

[Epoch: 92][#examples: 9600/46620][#steps: 64550]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 92][#examples: 12800/46620][#steps: 64600]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 92][#examples: 16000/46620][#steps: 64650]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 92][#examples: 19200/46620][#steps: 64700]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 92][#examples: 22400/46620][#steps: 64750]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 92][#examples: 25600/46620][#steps: 64800]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 92][#examples: 28800/46620][#steps: 64850]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 92][#examples: 32000/46620][#steps: 64900]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 92][#examples: 35200/46620][#steps: 64950]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 92][#examples: 38400/46620][#s

The number of correct pinyin predictions: 4292
Val Acc on Pinyin: 0.736
The number of correct predictions: 4222
[Eval End]: Current Teacher Forcing Ratio: 0.200

---------------------------------------
[Epoch: 94][Validatiing...]
	 patience: 0/20
	 Val. Loss: 1.101 | Val. Acc: 0.719 | Val. PPL:   3.008
	 BEST. Val. Loss: 0.488 | BEST. Val. Acc: 0.720 | Val. Loss: 1.090 | BEST. Val. Epoch: 50 | BEST. Val. Step: 35500
---------------------------------------

[Eval Start]: Current Teacher Forcing Ratio: 0.000
The number of correct pinyin predictions: 4315
Val Acc on Pinyin: 0.740
The number of correct predictions: 4189
[Eval End]: Current Teacher Forcing Ratio: 0.200

---------------------------------------
[Epoch: 94][Validatiing...]
	 patience: 0/20
	 Val. Loss: 1.135 | Val. Acc: 0.719 | Val. PPL:   3.111
	 BEST. Val. Loss: 0.488 | BEST. Val. Acc: 0.720 | Val. Loss: 1.090 | BEST. Val. Epoch: 50 | BEST. Val. Step: 35500
---------------------------------------

Epoch: 95 | Time: 1m 23s
	T

[Epoch: 97][#examples: 16000/46620][#steps: 68150]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 97][#examples: 19200/46620][#steps: 68200]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 97][#examples: 22400/46620][#steps: 68250]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 97][#examples: 25600/46620][#steps: 68300]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 97][#examples: 28800/46620][#steps: 68350]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 97][#examples: 32000/46620][#steps: 68400]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 97][#examples: 35200/46620][#steps: 68450]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
[Epoch: 97][#examples: 38400/46620][#steps: 68500]
	Train Loss: 0.000 | Train PPL:   1.000 | lr: 9.838e-08
-----val------
[Eval Start]: Current Teacher Forcing Ratio: 0.000
The number of correct pinyin predictions: 4315
Val Acc on Pinyin: 0.740
The nu

[Eval Start]: Current Teacher Forcing Ratio: 0.000
The number of correct pinyin predictions: 4315
Val Acc on Pinyin: 0.740
The number of correct predictions: 4189
[Eval End]: Current Teacher Forcing Ratio: 0.200

---------------------------------------
[Epoch: 99][Validatiing...]
	 patience: 0/20
	 Val. Loss: 1.135 | Val. Acc: 0.719 | Val. PPL:   3.111
	 BEST. Val. Loss: 0.488 | BEST. Val. Acc: 0.720 | Val. Loss: 1.090 | BEST. Val. Epoch: 50 | BEST. Val. Step: 35500
---------------------------------------

Epoch: 100 | Time: 1m 23s
	Train Loss: 0.000 | Train PPL:   1.000
	 Val. Loss: 1.135 | Val. Acc: 0.719 | Val. PPL:   3.111


In [ ]:
vars(test_data.examples[0])

In [32]:
exp_num = 1
model.load_state_dict(torch.load('experiments/exp' + str(exp_num) + '/acc-model-seq2seq.pt'))

valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, scheduler, is_test=False, multi_task=True)
test_loss, test_acc = evaluate(model, test_iterator, criterion, scheduler, is_test=True, multi_task=True)

# Note that the final translation accs might differ from below because of floating point error.
# But they should be the same in most of the cases.
print(f'| Valid Loss: {valid_loss:.3f} | Valid PPL: {math.exp(valid_loss):7.3f} | Valid ACC: {valid_acc:.3f}')
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} | Test ACC: {test_acc:.3f}')

[Eval Start]: Current Teacher Forcing Ratio: 0.000
The number of correct pinyin predictions: 4310
Val Acc on Pinyin: 0.740
The number of correct predictions: 4196
[Eval End]: Current Teacher Forcing Ratio: 0.200
[Eval Start]: Current Teacher Forcing Ratio: 0.000
The number of correct pinyin predictions: 4297
Val Acc on Pinyin: 0.737
The number of correct predictions: 4220
[Eval End]: Current Teacher Forcing Ratio: 0.200
| Valid Loss: 1.090 | Valid PPL:   2.975 | Valid ACC: 0.720
| Test Loss: 1.109 | Test PPL:   3.030 | Test ACC: 0.724


In [21]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):

    model.eval()
        
    # lower-cased
    tokens = [token.lower() for token in sentence]

    # add <sos> and <eos>
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    
    # vectorized
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)]).to(device)
    
    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor, src_len)
        hidden = torch.tanh(model.bridge(hidden))
        cell = None
        if model.encoder.rnn_type == 'LSTM':
            cell = torch.tanh(model.bridge_cell(model.encoder.cell))

    mask = model.create_mask(src_tensor)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
    
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs, mask, cell=cell)
            cell = model.decoder.cell

        attentions[i] = attention
            
        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    print(trg_tokens)
    
    return trg_tokens[1:], attentions[:len(trg_tokens)-1]

---------------
### Evaluation on the validation set (Customised)

In [22]:
def evaluate_trans(data, mode='TRG'):
    # Translate all valid set and report Accuracy
    result = pd.DataFrame(columns=["SRC", "PRED", "TRG"])
    i = 0
    for exp in data.examples:
        if mode == 'PINYIN':
            pred = translate_sentence(vars(exp)['src'], SRC, PINYIN_STR, model, device)[0][:-1]
            result.loc[str(i)] = (["".join(vars(exp)['src']), "".join(pred), "".join(vars(exp)['pinyin_str'])])
        else:
            pred = translate_sentence(vars(exp)['src'], SRC, TRG, model, device)[0][:-1]
            result.loc[str(i)] = (["".join(vars(exp)['src']), "".join(pred), "".join(vars(exp)['trg'])])
        i += 1
    # print(np.sum(result['PRED'] == result['TRG']))
    acc = np.sum(result['PRED'] == result['TRG']) / len(data.examples)
    return result, acc

In [23]:
result_valid, acc_valid = evaluate_trans(valid_data)
result_test, acc_test = evaluate_trans(test_data)
acc_valid, acc_test

['<sos>', '亨', '尼', '恩', '<eos>']
['<sos>', '特', '鲁', '尔', '森', '<eos>']
['<sos>', '马', '利', '亚', '德', '<eos>']
['<sos>', '希', '勒', '尔', '<eos>']
['<sos>', '埃', '德', '顿', '<eos>']
['<sos>', '索', '特', '<eos>']
['<sos>', '麦', '卡', '斯', '蒂', '<eos>']
['<sos>', '克', '利', '夫', '特', '<eos>']
['<sos>', '阿', '维', '洛', '<eos>']
['<sos>', '奥', '萨', '尔', '斯', '顿', '<eos>']
['<sos>', '艾', '尔', '温', '<eos>']
['<sos>', '库', '克', '曼', '<eos>']
['<sos>', '特', '尔', '福', '德', '<eos>']
['<sos>', '斯', '蒂', '克', '尼', '<eos>']
['<sos>', '西', '格', '斯', '<eos>']
['<sos>', '扎', '拉', '纳', '<eos>']
['<sos>', '厄', '尔', '<eos>']
['<sos>', '钱', '塞', '勒', '<eos>']
['<sos>', '皮', '尔', '斯', '<eos>']
['<sos>', '布', '拉', '德', '沃', '思', '<eos>']
['<sos>', '麦', '克', '贝', '思', '<eos>']
['<sos>', '莱', '西', '<eos>']
['<sos>', '洛', '克', '<eos>']
['<sos>', '戈', '伦', '伯', '格', '<eos>']
['<sos>', '弗', '里', '丹', '<eos>']
['<sos>', '布', '兰', '普', '顿', '<eos>']
['<sos>', '克', '莱', '勒', '<eos>']
['<sos>', '吉', '尔', '德', '赫', '斯', '<

['<sos>', '布', '兰', '南', '<eos>']
['<sos>', '戈', '斯', '克', '<eos>']
['<sos>', '格', '拉', '托', '<eos>']
['<sos>', '基', '韦', '特', '<eos>']
['<sos>', '贝', '思', '<eos>']
['<sos>', '卢', '克', '斯', '<eos>']
['<sos>', '比', '勒', '<eos>']
['<sos>', '斯', '托', '尼', '休', '尔', '<eos>']
['<sos>', '埃', '尔', '丝', '<eos>']
['<sos>', '威', '金', '斯', '<eos>']
['<sos>', '扎', '里', '亚', '<eos>']
['<sos>', '勒', '休', '<eos>']
['<sos>', '詹', '宁', '斯', '<eos>']
['<sos>', '托', '林', '顿', '<eos>']
['<sos>', '瓦', '格', '曼', '<eos>']
['<sos>', '富', '尔', '<eos>']
['<sos>', '马', '恩', '<eos>']
['<sos>', '切', '尼', '<eos>']
['<sos>', '科', '克', '林', '<eos>']
['<sos>', '马', '达', '姆', '<eos>']
['<sos>', '麦', '卡', '利', '克', '西', '<eos>']
['<sos>', '诺', '戈', '戈', '德', '<eos>']
['<sos>', '温', '伯', '恩', '<eos>']
['<sos>', '鲍', '兹', '曼', '<eos>']
['<sos>', '哈', '伯', '沙', '姆', '<eos>']
['<sos>', '朗', '斯', '登', '<eos>']
['<sos>', '马', '特', '恩', '<eos>']
['<sos>', '安', '策', '斯', '<eos>']
['<sos>', '英', '弗', '克', '莱', '莱', '德', '<eos>']

['<sos>', '惠', '滕', '<eos>']
['<sos>', '帕', '利', '泽', '<eos>']
['<sos>', '勒', '盖', '特', '<eos>']
['<sos>', '埃', '利', '<eos>']
['<sos>', '米', '林', '顿', '<eos>']
['<sos>', '布', '卢', '姆', '菲', '尔', '德', '<eos>']
['<sos>', '阿', '门', '特', '<eos>']
['<sos>', '甘', '特', '<eos>']
['<sos>', '马', '勒', '诺', '<eos>']
['<sos>', '帕', '迪', '尤', '<eos>']
['<sos>', '朗', '西', '曼', '<eos>']
['<sos>', '泰', '雷', '尔', '<eos>']
['<sos>', '赫', '施', '<eos>']
['<sos>', '德', '斯', '<eos>']
['<sos>', '勒', '弗', '<eos>']
['<sos>', '奥', '博', '尔', '<eos>']
['<sos>', '贝', '泽', '<eos>']
['<sos>', '埃', '尔', '津', '加', '<eos>']
['<sos>', '莫', '丁', '<eos>']
['<sos>', '阿', '方', '索', '<eos>']
['<sos>', '班', '普', '顿', '<eos>']
['<sos>', '辛', '科', '克', '斯', '<eos>']
['<sos>', '约', '卡', '施', '<eos>']
['<sos>', '图', '思', '<eos>']
['<sos>', '法', '克', '勒', '<eos>']
['<sos>', '弗', '斯', '特', '<eos>']
['<sos>', '特', '雷', '普', '<eos>']
['<sos>', '兹', '达', '尼', '斯', '<eos>']
['<sos>', '费', '特', '<eos>']
['<sos>', '莱', '格', '<eos>']
['<so

['<sos>', '坎', '塔', '菲', '奥', '<eos>']
['<sos>', '布', '赖', '奥', '尼', '斯', '<eos>']
['<sos>', '布', '拉', '德', '古', '德', '<eos>']
['<sos>', '阿', '多', '<eos>']
['<sos>', '罗', '森', '菲', '尔', '德', '<eos>']
['<sos>', '本', '托', '内', '利', '<eos>']
['<sos>', '霍', '利', '斯', '顿', '<eos>']
['<sos>', '托', '利', '特', '<eos>']
['<sos>', '麦', '卡', '洛', '<eos>']
['<sos>', '埃', '伯', '哈', '特', '<eos>']
['<sos>', '古', '利', '亚', '诺', '斯', '<eos>']
['<sos>', '达', '凯', '恩', '<eos>']
['<sos>', '罗', '德', '曼', '<eos>']
['<sos>', '沃', '特', '莫', '尔', '<eos>']
['<sos>', '奥', '加', '勒', '<eos>']
['<sos>', '塔', '尔', '本', '<eos>']
['<sos>', '克', '莱', '顿', '<eos>']
['<sos>', '赫', '姆', '斯', '特', '里', '特', '<eos>']
['<sos>', '艾', '科', '尔', '茨', '<eos>']
['<sos>', '布', '尔', '蒂', '迪', '德', '<eos>']
['<sos>', '布', '罗', '德', '<eos>']
['<sos>', '博', '比', '特', '<eos>']
['<sos>', '韦', '普', '尔', '<eos>']
['<sos>', '斯', '特', '里', '克', '曼', '<eos>']
['<sos>', '奥', '尔', '<eos>']
['<sos>', '威', '尔', '曼', '<eos>']
['<sos>', '米', '德', '

['<sos>', '沃', '思', '<eos>']
['<sos>', '哈', '格', '<eos>']
['<sos>', '格', '尔', '泽', '<eos>']
['<sos>', '波', '尔', '顿', '<eos>']
['<sos>', '基', '比', '<eos>']
['<sos>', '埃', '林', '斯', '塔', '德', '<eos>']
['<sos>', '布', '劳', '泽', '<eos>']
['<sos>', '费', '尔', '森', '<eos>']
['<sos>', '莫', '纳', '特', '<eos>']
['<sos>', '格', '林', '达', '尔', '<eos>']
['<sos>', '康', '福', '德', '<eos>']
['<sos>', '西', '勒', '里', '<eos>']
['<sos>', '邓', '克', '利', '夫', '<eos>']
['<sos>', '金', '南', '<eos>']
['<sos>', '菲', '亚', '尔', '科', '<eos>']
['<sos>', '托', '因', '<eos>']
['<sos>', '巴', '克', '温', '<eos>']
['<sos>', '德', '拉', '戈', '<eos>']
['<sos>', '康', '罗', '伊', '<eos>']
['<sos>', '吉', '伦', '<eos>']
['<sos>', '马', '尔', '格', '雷', '姆', '<eos>']
['<sos>', '本', '海', '默', '<eos>']
['<sos>', '布', '拉', '肯', '里', '德', '<eos>']
['<sos>', '克', '勒', '菲', '尔', '德', '<eos>']
['<sos>', '布', '雷', '登', '<eos>']
['<sos>', '科', '洛', '顿', '<eos>']
['<sos>', '博', '托', '尔', '夫', '森', '<eos>']
['<sos>', '韦', '兰', '<eos>']
['<sos>', '沃', '尔'

...

['<sos>', '阿', '克', '斯', '顿', '<eos>']
['<sos>', '劳', '克', '斯', '<eos>']
['<sos>', '庞', '彻', '<eos>']
['<sos>', '金', '贡', '<eos>']
['<sos>', '哈', '德', '塞', '尔', '<eos>']
['<sos>', '切', '斯', '诺', '夫', '<eos>']
['<sos>', '德', '图', '诺', '<eos>']
['<sos>', '海', '森', '伯', '格', '<eos>']
['<sos>', '欣', '德', '<eos>']
['<sos>', '科', '尔', '伯', '格', '<eos>']
['<sos>', '麦', '高', '恩', '<eos>']
['<sos>', '奥', '尔', '<eos>']
['<sos>', '泽', '利', '科', '<eos>']
['<sos>', '普', '里', '比', '森', '<eos>']
['<sos>', '休', '梅', '克', '<eos>']
['<sos>', '鲁', '赫', '<eos>']
['<sos>', '科', '勒', '<eos>']
['<sos>', '布', '拉', '特', '<eos>']
['<sos>', '斯', '沃', '特', '利', '<eos>']
['<sos>', '索', '伊', '莎', '<eos>']
['<sos>', '伯', '多', '<eos>']
['<sos>', '奇', '尔', '德', '豪', '斯', '<eos>']
['<sos>', '鲍', '特', '尔', '<eos>']
['<sos>', '勒', '克', '<eos>']
['<sos>', '克', '理', '查', '德', '<eos>']
['<sos>', '拉', '德', '克', '利', '夫', '<eos>']
['<sos>', '麦', '卡', '尔', '平', '<eos>']
['<sos>', '莫', '格', '里', '奇', '<eos>']
['<sos>', '波', '斯'

['<sos>', '布', '德', '罗', '<eos>']
['<sos>', '卡', '拉', '德', '<eos>']
['<sos>', '斯', '特', '罗', '恩', '<eos>']
['<sos>', '卡', '利', '克', '斯', '特', '斯', '<eos>']
['<sos>', '金', '特', '<eos>']
['<sos>', '格', '林', '杰', '<eos>']
['<sos>', '麦', '加', '恩', '<eos>']
['<sos>', '赫', '彭', '斯', '特', '尔', '<eos>']
['<sos>', '盖', '卡', '伦', '<eos>']
['<sos>', '米', '利', '奥', '拉', '<eos>']
['<sos>', '尼', '塔', '<eos>']
['<sos>', '肖', '布', '<eos>']
['<sos>', '利', '克', '<eos>']
['<sos>', '海', '因', '里', '希', '<eos>']
['<sos>', '乔', '姆', '利', '<eos>']
['<sos>', '戈', '梅', '里', '<eos>']
['<sos>', '利', '肯', '<eos>']
['<sos>', '赫', '尔', '伯', '德', '<eos>']
['<sos>', '阿', '多', '科', '克', '斯', '<eos>']
['<sos>', '索', '尔', '<eos>']
['<sos>', '托', '林', '顿', '<eos>']
['<sos>', '亨', '博', '<eos>']
['<sos>', '特', '鲁', '珀', '<eos>']
['<sos>', '米', '金', '<eos>']
['<sos>', '兰', '德', '西', '<eos>']
['<sos>', '伊', '贝', '奈', '尔', '斯', '<eos>']
['<sos>', '阿', '雷', '诺', '维', '茨', '<eos>']
['<sos>', '维', '弗', '斯', '<eos>']
['<sos>', '雷'

['<sos>', '阿', '蒂', '斯', '<eos>']
['<sos>', '埃', '米', '莉', '<eos>']
['<sos>', '麦', '克', '雷', '<eos>']
['<sos>', '泰', '特', '尔', '<eos>']
['<sos>', '欣', '塔', '格', '<eos>']
['<sos>', '梅', '利', '亚', '<eos>']
['<sos>', '贝', '克', '曼', '<eos>']
['<sos>', '克', '里', '斯', '蒂', '<eos>']
['<sos>', '罗', '明', '<eos>']
['<sos>', '科', '克', '汉', '斯', '<eos>']
['<sos>', '赖', '弗', '尔', '<eos>']
['<sos>', '维', '科', '伊', '<eos>']
['<sos>', '坎', '波', '<eos>']
['<sos>', '奥', '斯', '特', '<eos>']
['<sos>', '阿', '斯', '托', '里', '亚', '<eos>']
['<sos>', '基', '吉', '<eos>']
['<sos>', '辛', '基', '纳', '<eos>']
['<sos>', '麦', '吉', '奇', '<eos>']
['<sos>', '克', '伦', '肖', '<eos>']
['<sos>', '吉', '斯', '金', '<eos>']
['<sos>', '施', '里', '姆', '斯', '利', '<eos>']
['<sos>', '勒', '利', '弗', '<eos>']
['<sos>', '谢', '林', '<eos>']
['<sos>', '里', '马', '<eos>']
['<sos>', '莫', '里', '什', '<eos>']
['<sos>', '斯', '托', '弗', '尔', '<eos>']
['<sos>', '斯', '泰', '内', '茨', '<eos>']
['<sos>', '福', '斯', '特', '<eos>']
['<sos>', '莱', '<eos>']
['<sos>',

['<sos>', '阿', '马', '利', '<eos>']
['<sos>', '多', '兰', '<eos>']
['<sos>', '卡', '勒', '斯', '<eos>']
['<sos>', '惠', '奇', '科', '特', '<eos>']
['<sos>', '马', '修', '斯', '<eos>']
['<sos>', '迪', '马', '<eos>']
['<sos>', '内', '塞', '萨', '里', '<eos>']
['<sos>', '伯', '格', '斯', '<eos>']
['<sos>', '赫', '兰', '德', '<eos>']
['<sos>', '格', '雷', '<eos>']
['<sos>', '雷', '维', '斯', '<eos>']
['<sos>', '本', '迪', '克', '森', '<eos>']
['<sos>', '加', '普', '<eos>']
['<sos>', '基', '姆', '伦', '<eos>']
['<sos>', '法', '格', '斯', '<eos>']
['<sos>', '福', '克', '<eos>']
['<sos>', '斯', '普', '拉', '德', '利', '<eos>']
['<sos>', '奇', '斯', '特', '<eos>']
['<sos>', '莫', '比', '<eos>']
['<sos>', '科', '尔', '特', '林', '<eos>']
['<sos>', '康', '尼', '特', '<eos>']
['<sos>', '梅', '特', '<eos>']
['<sos>', '斯', '卡', '丁', '<eos>']
['<sos>', '巴', '金', '特', '<eos>']
['<sos>', '斯', '滕', '威', '格', '<eos>']
['<sos>', '赖', '特', '迈', '尔', '<eos>']
['<sos>', '梅', '切', '恩', '<eos>']


(0.7199725463280714, 0.7240905971173645)

In [26]:
num = 1
result_valid.to_excel('experiments/' + 'exp' + str(num) + '/valid_result' + '.xlsx', index=False)
result_test.to_excel('experiments/' + 'exp' + str(num) + '/test_result' + '.xlsx', index=False)

In [29]:
with open('experiments/' + 'exp' + str(num) + '/acc.txt', 'w+', encoding='utf-8') as f:
    f.write('Valid Accuracy: {}\n'.format(acc_valid))
    f.write('Test Accuracy: {}\n'.format(acc_test))
    # f.write('Test Accuracy (Multi): {}\n'.format(acc_test_multi))
    f.write('Test Loss: {}\n'.format(test_loss))

with open('experiments/' + 'exp' + str(num) + '/setting.txt', 'w+', encoding='utf-8') as f:
    f.write('Params #: {}\nMulti-task ratio on main task: {}\n{}'.format(count_parameters(model), multi_task_ratio, show_params))

In [28]:
with open('experiments/' + 'exp' + str(num) + '/valid_pred.txt', 'w+', encoding='utf-8') as f:
    for p in result_valid['PRED']:
        f.write(' '.join(p) + '\n')
        
with open('experiments/' + 'exp' + str(num) + '/valid_ref.txt', 'w+', encoding='utf-8') as f:
    for p in result_valid['TRG']:
        f.write(' '.join(p) + '\n')
        
with open('experiments/' + 'exp' + str(num) + '/test_pred.txt', 'w+', encoding='utf-8') as f:
    for p in result_test['PRED']:
        f.write(' '.join(p) + '\n')
        
with open('experiments/' + 'exp' + str(num) + '/test_ref.txt', 'w+', encoding='utf-8') as f:
    for p in result_test['TRG']:
        f.write(' '.join(p) + '\n')

-----------